### GLOBAL CONFIG

In [110]:
%load_ext autoreload
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

PREPROCESS = True # Do a fresh preprocess
MAKE_NEW_EMBEDDING = True # If False, the stored one will be loaded
# EMB_MAX_WORDS = None
RANDOM_SEED = 456
SAVE_TRAINED_MODEL = True

PREPROCESS_INPUT = './data/training.1600000.processed.noemoticon.csv'
PREPROCESS_OUTPUT = './data/preprocessed.csv'
GLOVE_FILE = './data/glove.6B.50d.txt'
EMB_PKL = './models/emb_layer.pkl'
MODEL_PKL = './models/model.pkl'
MODEL_TYPE = 'sequential'
model_path = 'models/trained_'+ MODEL_TYPE
TWEETS_TO_EXPLAIN = [111, 156, 933487, 933565]

new_model = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Preprocess data and store it

In [ ]:
from pre import preprocess

if PREPROCESS:
    preprocess(i=PREPROCESS_INPUT, o=PREPROCESS_OUTPUT, slice=1000)

### Load preprocessed data

In [ ]:
from parse import load_data
from sklearn.model_selection import train_test_split

data = load_data(PREPROCESS_OUTPUT)
X = data['tweet']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)

y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)
y_val = tf.keras.utils.to_categorical(y_val, 2)


In [93]:
### Create Empty model
%autoreload 2

from TextClassifierModel import new_classifier, load_classifier, save_classifier






if new_model:
    model_type = "recurrent"
    text_classifier = new_classifier(glove_file = GLOVE_FILE, data=data, model_type=model_type)
    save_classifier(text_classifier, 'models/untrained')
else:
    text_classifier = load_classifier(model_path=model_path)
print(text_classifier.model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           140400    
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1280256   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1

### Train

In [ ]:
%autoreload 2
from evaluation import plot_history

history = text_classifier.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=60, epochs=30, verbose=1)
plot_history(history)


### Save trained model

In [ ]:
from TextClassifierModel import save_classifier
save_classifier(text_classifier, model_path)

### Evaluate production (forbidden during tuning)

In [ ]:
from evaluation import evaluate_model

evaluate_model(text_classifier, X_test, y_test, verbose=False)

### Explain prediction

In [111]:
# Load unprocessed data for readability
from pre import load_unprocessed
df_orig = load_unprocessed(PREPROCESS_INPUT)




In [112]:
# Fetch subset of training tweets
%autoreload 2
from pre import split_and_preprocess
indices = [111, 156, 933487, 933565]

explain_tweets_orig, explain_tweets_prep = split_and_preprocess(df_orig, TWEETS_TO_EXPLAIN)



100%|██████████| 4/4 [00:00<00:00, 663.66it/s]


In [113]:
from explainability import explain_and_save
explain_and_save(explain_tweets_orig, explain_tweets_prep, TWEETS_TO_EXPLAIN, text_classifier,  MODEL_TYPE)


mean_KL_divergence 0.0024789303535317424
score 1.0
mean_KL_divergence 0.0009783858683550606
score 0.9568860590655804
mean_KL_divergence 0.0014899328141211635
score 0.7429565291074613
mean_KL_divergence 0.00214878942366003
score 1.0


In [114]:
#Show prediction for single tweet:
tweet_idx = TWEETS_TO_EXPLAIN[3]
te = TextExplainer(random_state=42)
te.fit(explain_tweets_prep[tweet_idx], text_classifier.predict_proba)
te.show_prediction()

In [ ]:

from visualize_embeddings import  display_pca_scatter_plot
display_pca_scatter_plot(GLOVE_FILE)